In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from time import time
import numpy as np

In [2]:
torch.manual_seed(1234)

In [3]:
def square_distance(src, dst):
    """
    Calculate Euclid distance between each two points.
    计算每两个点之间的欧几里得距离。
    dist = (xn - xm)^2 + (yn - ym)^2 + (zn - zm)^2
         = sum(src**2,dim=-1)+sum(dst**2,dim=-1)-2*src^T*dst
    Input:
        src: source points, [B, N, C]
        dst: target points, [B, M, C]
    Output:
        dist: per-point square distance, [B, N, M]
    """
    return torch.sum((src[:, :, None] - dst[:, None]) ** 2, dim=-1)

def index_points(points, idx):
    """
    Input:
        points: input points data, [B, N, C]
        idx: sample index data, [B, S, [K]]
    Return:
        new_points:, indexed points data, [B, S, [K], C]
    """
    raw_size = idx.size()
    idx = idx.reshape(raw_size[0], -1)
    res = torch.gather(points, 1, idx[..., None].expand(-1, -1, points.size(-1)))
    return res.reshape(*raw_size, -1)

In [4]:
d_model=512
d_points = 32
fc1 = nn.Linear(d_points, d_model)
fc2 = nn.Linear(d_model, d_points)
fc_delta = nn.Sequential(
    nn.Linear(3, d_model),
    nn.ReLU(),
    nn.Linear(d_model, d_model)
)
fc_gamma = nn.Sequential(
    nn.Linear(d_model, d_model),
    nn.ReLU(),
    nn.Linear(d_model, d_model)
)
w_qs = nn.Linear(d_model, d_model, bias=False)
w_ks = nn.Linear(d_model, d_model, bias=False)
w_vs = nn.Linear(d_model, d_model, bias=False)
k = 16

# start

In [5]:
random_seed = 1234
torch.manual_seed(random_seed)
point = torch.randn(8,1024,6)
xyz = point[..., :3]
print(xyz.shape)

torch.Size([8, 1024, 3])


In [6]:
fcc1 = nn.Sequential(
    nn.Linear(6, 32), 
    nn.ReLU(),
    nn.Linear(32, 32) # point [16,1024,6] to [16,1024,32]
    )
features=fcc1(point)
print(features.shape)

torch.Size([8, 1024, 32])


In [7]:
dists = square_distance(xyz, xyz)# 计算点距离 逐个相减
    #dist = (xn - xm)^2 + (yn - ym)^2 + (zn - zm)^2
    #     = sum(src**2,dim=-1)+sum(dst**2,dim=-1)-2*src^T*dst
print(dists.shape)

torch.Size([8, 1024, 1024])


In [8]:
knn_idx = dists.argsort()[:, :, :k]  # b x n x k 排序取前k个
print(knn_idx.shape)

torch.Size([8, 1024, 16])


In [9]:
knn_xyz = index_points(xyz, knn_idx)
print(knn_xyz.shape)

torch.Size([8, 1024, 16, 3])


In [10]:
pre = features
x = fc1(features)   #features [16,1024,32] to [16,1024,512]
print(x.shape)
q, k, v = w_qs(x), index_points(w_ks(x), knn_idx), index_points(w_vs(x), knn_idx)
print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([8, 1024, 512])
torch.Size([8, 1024, 512])
torch.Size([8, 1024, 16, 512])
torch.Size([8, 1024, 16, 512])


In [11]:
k.size(-1)

512

In [12]:
np.sqrt(k.size(-1))

22.627416997969522

In [13]:
pos_enc = fc_delta(xyz[:, :, None] - knn_xyz)  # b x n x k x f  邻居向量
print(pos_enc.shape)
print((xyz[:, :, None] - knn_xyz).shape)

torch.Size([8, 1024, 16, 512])
torch.Size([8, 1024, 16, 3])


In [14]:
attn = fc_gamma(q[:, :, None] - k + pos_enc)
print(attn.shape)
attn = F.softmax(attn / np.sqrt(k.size(-1)), dim=-2)  # b x n x k x f
print(attn.shape)

torch.Size([8, 1024, 16, 512])
torch.Size([8, 1024, 16, 512])


In [15]:
res = torch.einsum('bmnf,bmnf->bmf', attn, v + pos_enc)
print(res.shape)
res = fc2(res) + pre
print(res.shape)

torch.Size([8, 1024, 512])
torch.Size([8, 1024, 32])


In [ ]:
torch.manual_seed(1234)
point1 = torch.randn(2,4,3)
print(point1)
print(point1[:, :, None].shape)
# print(point[:, :, None])
print(point1[:, None].shape)
# print(point[:, None])
print((point1[:, :, None] - point1[:, None]).shape)
# print(point[:, :, None] - point[:, None])
print(torch.sum((point1[:, :, None] - point1[:, None]) ** 2, dim=-1).shape)

In [2]:
torch.manual_seed(1234)
x = torch.randn(2,4,3,2)
x

tensor([[[[-0.1117, -0.4966],
          [ 0.1631, -0.8817],
          [ 0.0539,  0.6684]],

         [[-0.0597, -0.4675],
          [-0.2153,  0.8840],
          [-0.7584, -0.3689]],

         [[-0.3424, -1.4020],
          [ 0.3206, -1.0219],
          [ 0.7988, -0.0923]],

         [[-0.7049, -1.6024],
          [ 0.2891,  0.4899],
          [-0.3853, -0.7120]]],


        [[[-0.1706, -1.4594],
          [ 0.2207,  0.2463],
          [-1.3248,  0.6970]],

         [[-0.6631,  1.2158],
          [-1.4949,  0.8810],
          [-1.1786, -0.9340]],

         [[-0.5675, -0.2772],
          [-2.1834,  0.3668],
          [ 0.9380,  0.0078]],

         [[-0.3139, -1.1567],
          [ 1.8409, -1.0174],
          [ 1.2192,  0.1601]]]])

In [3]:
x = torch.max(x, 2)[0]

In [4]:
x

tensor([[[ 0.1631,  0.6684],
         [-0.0597,  0.8840],
         [ 0.7988, -0.0923],
         [ 0.2891,  0.4899]],

        [[ 0.2207,  0.6970],
         [-0.6631,  1.2158],
         [ 0.9380,  0.3668],
         [ 1.8409,  0.1601]]])

# BN

In [3]:
# With Learnable Parameters
m = nn.BatchNorm1d(4)
# Without Learnable Parameters
# m = nn.BatchNorm1d(4, affine=False)

In [2]:
input = torch.randn(2, 4, 3)
input

tensor([[[ 1.3339e+00, -1.2772e+00,  1.5149e+00],
         [ 1.2043e+00,  8.9255e-05,  4.7879e-01],
         [-3.8901e-01,  3.0105e-03,  8.9851e-01],
         [-2.1315e+00, -1.1159e-01,  3.0682e-01]],

        [[-6.6094e-01, -5.6356e-01,  2.0013e+00],
         [-5.1532e-01,  1.1222e+00,  5.8444e-01],
         [ 1.1154e+00, -1.7638e+00, -2.7672e-01],
         [-1.3785e+00,  1.9836e-01, -6.8913e-01]]])

In [ ]:
output = m(input)
output

# 池化

In [10]:
import torch
import torch.nn as nn
a=torch.ones(2,3,4)
a[0,1,2]=0

In [11]:
a

tensor([[[1., 1., 1., 1.],
         [1., 1., 0., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])

In [14]:
nn.AdaptiveAvgPool1d(5)(a)

tensor([[[1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 0.5000, 0.5000, 1.0000],
         [1.0000, 1.0000, 1.0000, 1.0000, 1.0000]],

        [[1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000, 1.0000, 1.0000]]])

In [15]:
nn.AdaptiveAvgPool1d(5)(a).shape

torch.Size([2, 3, 5])

In [27]:
b = nn.AdaptiveAvgPool1d(1)(a)
b

tensor([[[1.0000],
         [0.7500],
         [1.0000]],

        [[1.0000],
         [1.0000],
         [1.0000]]])

In [32]:
b.shape

torch.Size([2, 3, 1])

In [41]:
b.squeeze(-1).shape

torch.Size([2, 3])

In [40]:
k = nn.Flatten(b)
k

Flatten(
  start_dim=tensor([[[1.0000],
           [0.7500],
           [1.0000]],
  
          [[1.0000],
           [1.0000],
           [1.0000]]]), end_dim=-1
)

In [33]:
b.view(2,3).shape

torch.Size([2, 3])

In [17]:
nn.AdaptiveAvgPool1d(2)(a).shape

torch.Size([2, 3, 2])

## max

In [46]:
input = torch.randn(2, 2, 2,3)
input

tensor([[[[ 3.1482e-01,  1.3892e-02, -1.1193e+00],
          [ 1.0446e+00, -7.3216e-01, -1.6389e-05]],

         [[ 5.4959e-01, -1.1447e+00,  5.2245e-01],
          [ 5.9504e-01,  8.2466e-02,  6.9690e-01]]],


        [[[-9.7726e-01,  4.2012e-01, -6.1108e-01],
          [ 4.2146e-01,  1.5944e+00,  2.2480e+00]],

         [[-1.0624e+00, -8.8576e-01,  1.0112e+00],
          [ 1.1669e+00, -8.0493e-01,  1.6379e+00]]]])

In [43]:
pool = nn.MaxPool1d(5)

In [48]:
for k in input:
    print(k.shape)

torch.Size([2, 2, 3])
torch.Size([2, 2, 3])


In [ ]:
output = pool(input)
output

In [6]:
import torch
import torch.nn as nn
from torch.autograd import Variable

m = nn.MaxPool2d((1,3))
input = Variable(torch.randn(2,2, 2, 3))
output = m(input)

print(input)
print(output)

tensor([[[[-0.9360, -1.1713,  0.3245],
          [-0.5693,  0.0517,  1.3003]],

         [[ 0.0128, -0.0502, -0.8512],
          [-0.6419, -1.2006,  0.5811]]],


        [[[-0.2277, -0.7041,  0.1951],
          [-0.3848, -1.7010,  0.4325]],

         [[-1.1343,  0.0304, -0.7306],
          [ 2.1412, -0.4824,  0.9183]]]])
tensor([[[[0.3245],
          [1.3003]],

         [[0.0128],
          [0.5811]]],


        [[[0.1951],
          [0.4325]],

         [[0.0304],
          [2.1412]]]])


In [8]:
for j in range(2):
    print(j)

0
1
